In [ ]:
#---- if not installed ----

#pip install --upgrade PyQt5 PyQt5-sip
#pip install PyQtWebEngine
#pip install --upgrade PyQt5 PyQt5-sip PyQtWebEngine

In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re
import tempfile

from math import pi
import plotly.express as px
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar,
)

from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import *
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEngineSettings
from PyQt5.uic import loadUi
from PyQt5.QtCore import *
from PyQt5.QtGui import QPixmap

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import AgglomerativeClustering

from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean

In [2]:
# --------------------------------------------------------------------------------------------------------------------------
# Defining global variables (needed later)


metric_features = ['vendor_count', 'product_count', 'is_chain',
       'first_order', 'last_order', 'lifetime_days', 'total_expenses',
       'avg_per_product', 'avg_per_order', 'avg_order_size',
       'culinary_variety', 'chain_preference', 'loyalty_to_venders', 'customer_age_group']


#for scaling:
metric_features_scale = ['lifetime_days', 'total_expenses', 'avg_per_product', 'avg_per_order', 'avg_order_size',
       'culinary_variety', 'chain_preference', 'loyalty_to_venders', 'customer_age_group']
group1 = ['first_order', 'last_order']
group2 = ['is_chain', 'vendor_count', 'product_count']

#values for late calculation of final values
last_order = 0
first_order = 0
lifetime_days = 0
total_expenses = 0
avg_per_product = 0
avg_per_order = 0
avg_order_size = 0
culinary_variety = 0
chain_preference = 0
loyalty_to_venders = 0
customer_age_group = 0
vendor_count = 0
product_count = 0
is_chain = 0
OrdersPlaced = 0


#saved labels, scalers, clusters and df
labels = np.loadtxt('../dataInterface/FINAL_hc_labels6.csv', delimiter=',', dtype=int)
clusters = joblib.load('../dataInterface/FINAL_clusters_hc6.pkl')

scaler1 = joblib.load('../dataInterface/scalerGroupBasic.pkl')
scaler2 = joblib.load('../dataInterface/scalerGroup1.pkl')
scaler3 = joblib.load('../dataInterface/scalerGroup2.pkl')

cluster_ids_set = list(map(int, sorted(set(labels))))

df = pd.read_csv("../dataInterface/DfScaling_Updatet.csv")
df_concat = pd.concat([df[metric_features],pd.Series(labels, name='labels', index=df.index)],axis=1)


# global variables for functionality
is_growing = True
lastGraph2 = 0
lastGraph3 = 0
lastGraph4 = 0
lastGraphCheck3 = 0

#dictionaries used later
warn_values = {
    "TT1_input": False, "TT2_input": False, "TT3_input": False, "TT4_input": False, "TT5_input": False, "TT6_slider": False, "TT9_input": False, # if there is a warning for unsual values
}

confirmed_status = {
    "TT1_input": False, "TT2_input": False, "TT3_input": False, "TT4_input": False, "TT5_input": False, "TT6_slider_label": False,
    "TT7_slider_label": False, "TT8_combo": False,"TT9_input": False,
}


In [3]:
# ----------------------------------------------------------------------------------------------------------------------------
# INFO: similar concepts are only explained once, so not at every button there are comments, just at the first one for example
# ----------------------------------------------------------------------------------------------------------------------------

class Ui_MainWindow(object):

    def apply_stylesheet(self, widget, border_color="white", focus_border_color="blue", border_px="2px",border_status="solid"):  # method to change stylesheet
        widget_name = widget.objectName()
        if "input" in widget_name:  # if it is input field
            widget.setStyleSheet(f"""
                QLineEdit {{
                    background-color: white;
                    color: black;
                    border: {border_px} {border_status} {border_color};
                    border: {border_px} {border_status} {border_color};
                    font-size: 16px;
                    padding: 5px;
                    border-radius: 10px;
                }}
                QLineEdit:focus {{
                    border: {border_px} {border_status} {focus_border_color};
                }}
            """)
        elif "combo" in widget_name:  # if it is combobox
            widget.setStyleSheet(f"""
                            background-color: white;
                            color: black;
                            border: {border_px} {border_status} {border_color};
                            font-size: 16px;
                            padding: 5px;
                            border-radius: 10px;
                    """)
            widget.setStyleSheet(f"""
                                    QComboBox {{
                                            background-color: white;
                                            color: black;
                                            border: {border_px} {border_status} {border_color};
                                            font-size: 16px;
                                            padding: 5px;
                                            border-radius: 10px;
                                        }}
                                    QComboBox:focus {{
                                        border: 2px solid blue;
                                    }}
                                    QComboBox:editable:on {{
                                        border: 2px solid blue;
                                    }}
                                    QComboBox:!editable:on {{
                                        border: 2px solid blue;
                                    }}
                                    QComboBox QAbstractItemView {{
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        selection-background-color: lightblue;
                                        selection-color: black;
                                    }}
                                                """)
        elif "slide" in widget_name:   # if it is a slider
            widget.setStyleSheet(f"""
                        QLabel {{
                            font-size: 16px;
                            color: black;
                            background-color: white;
                            border: {border_px} {border_status} {border_color};
                            border-radius: 5px;
                        }}
                    """)
        elif "button" in widget_name:  # if it is a button (only for certain buttons from page 3)
            if(widget_name == "button_3_1" or widget_name == "button_3_4"):
                widget.setStyleSheet(f"""
                                QPushButton {{
                                    font-size: 22px;
                                    background: qlineargradient(
                                            x1: 0, y1: 0, x2: 1, y2: 1,
                                            stop: 0 cornflowerblue,
                                            stop: 1 darkblue
                                    );
                                    border: 3px solid {border_color};
                                    border-radius: 25px;
                                    color: white;
                                   }}
                                QPushButton:pressed {{
                                    font-size: 22px;
                                    background: qlineargradient(
                                            x1: 0, y1: 0, x2: 1, y2: 1,
                                            stop: 0 cornflowerblue,
                                            stop: 1 darkblue
                                    );
                                    border: 8px solid {border_color};
                                    border-radius: 25px;
                                    color: white;
                                }}
                                   """)
            elif(widget_name == "button_3_2"):
                widget.setStyleSheet(f"""
                                    QPushButton {{
                                        font-size: 14px;
                                        background: qlineargradient(
                                                x1: 0, y1: 0, x2: 1, y2: 1,
                                                stop: 0 cornflowerblue,
                                                stop: 1 darkblue
                                        );
                                        border: 3px solid {border_color};
                                        border-radius: 25px;
                                        color: white;
                                       }}
                                    QPushButton:pressed {{
                                        font-size: 14px;
                                        background: qlineargradient(
                                                x1: 0, y1: 0, x2: 1, y2: 1,
                                                stop: 0 cornflowerblue,
                                                stop: 1 darkblue
                                        );
                                        border: 8px solid {border_color};
                                        border-radius: 25px;
                                        color: white;
                                    }}
                                       """)
            elif (widget_name == "button_3_3"):
                widget.setStyleSheet(f"""
                                    QPushButton {{
                                        font-size: 16px;
                                        background: qlineargradient(
                                                x1: 0, y1: 0, x2: 1, y2: 1,
                                                stop: 0 cornflowerblue,
                                                stop: 1 darkblue
                                        );
                                        border: 3px solid {border_color};
                                        border-radius: 25px;
                                        color: white;
                                       }}
                                    QPushButton:pressed {{
                                        font-size: 16px;
                                        background: qlineargradient(
                                                x1: 0, y1: 0, x2: 1, y2: 1,
                                                stop: 0 cornflowerblue,
                                                stop: 1 darkblue
                                        );
                                        border: 8px solid {border_color};
                                        border-radius: 25px;
                                        color: white;
                                    }}
                                       """)

    def on_combobox_change(self, combo_name,error_label_name):  # method for when there is a change in combobox, to reset its border and unconfirm it
        global confirmed_status
        combo_box = getattr(self, combo_name)
        error_label = getattr(self, error_label_name)
        combo_box.setStyleSheet("""
                                QComboBox {
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                QComboBox:focus {
                                    border: 2px solid blue;
                                }
                                QComboBox:editable:on {
                                    border: 2px solid blue;
                                }
                                QComboBox:!editable:on {
                                    border: 2px solid blue;
                                }
                                QComboBox QAbstractItemView {
                                    background-color: white;
                                    color: black;
                                    border: 2px solid white;
                                    selection-background-color: lightblue;
                                    selection-color: black;
                                }
                                            """)
        error_label.setVisible(False)
        confirmed_status[combo_name] = False

    def on_combobox_change_2(self, combo_name):   # method for changes on combobox to call again the last called graph, but with new value
        combo_box = getattr(self, combo_name)
        global lastGraph2
        global lastGraph3
        global lastGraph4
        if combo_name == "P2_combo":   #if the change comes from combobox on page 2
            if (lastGraph2 == 1):       #if last graph called was graph 1
                self.plot_2_1()     # call plot 1 again
            if (lastGraph2 == 2):
                self.plot_2_2()
            if (lastGraph2 == 3):
                self.plot_2_3()
        if combo_name == "P3_combo":    #if the change comes from combobox on page 3
            if (lastGraph3 == 1):
                self.plot_3_3()
        if combo_name == "P4_combo":    #if the change comes from combobox on page 4
            if (lastGraph4 == 1):
                self.plot_4_6()


    def on_text_change_input(self, label_name,input_name):  # method for when there is a change in inputfield, to reset its border and unconfirm it)
        global confirmed_status
        input = getattr(self, input_name)
        self.apply_stylesheet(input, border_color="white")
        confirmed_status[input_name] = False

    def on_checkbox_changed(self):   # method for when there is a change in checkbox, to call the last called plot again but with new checkbox values
        global lastGraphCheck3
        checked_count = 0
        if lastGraphCheck3 == 2:      # if last called graph was graph 2
            if self.C0_checkbox.isChecked():     # count which and how many checkboxes are checked
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count != 2):     # if more or less then 2 are checked (this graph works with exactly 2 only)
                return
            else:
                self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")   # else call the plot and assure that border has normal color
                self.plot_3_2()

        elif lastGraphCheck3 == 1:    # if last called graph was 1
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):  # if no box checked, do nothing
                return
            else:
                self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")   # else call the plot and reset border
                self.plot_3_1()

        elif lastGraphCheck3 == 3:
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):
                return
            else:
                self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
                self.plot_3_3()

        elif lastGraphCheck3 == 4:
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):
                return
            else:
                self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
                self.plot_3_4()



    def update_label_slide_TT6(self, value):       # when there is a update in the label slider TT6
        global confirmed_status
        self.TT6_slider_label.setText(f"{value}")      # set the text of the slider label to the current value of slider
        self.TT6_slider_label.setStyleSheet("""
                            QLabel {
                                font-size: 16px;
                                color: black;
                                background-color: white;
                                border: 2px solid gray;
                                border-radius: 5px;
                            }
                        """)
        confirmed_status["TT6_slider_label"] = False     # unconfirm the slider, because new value -> has to be confirmed again

    def update_label_slide_TT7(self, value):   # when there is a update in the label slider TT6
        global confirmed_status
        value = value / 100   # to show value as percent, because slider does not work with values lower than 1
        self.TT7_slider_label.setText(f"{value}")
        self.TT7_slider_label.setStyleSheet("""
                                    QLabel {
                                        font-size: 16px;
                                        color: black;
                                        background-color: white;
                                        border: 2px solid gray;
                                        border-radius: 5px;
                                    }
                                """)
        confirmed_status["TT7_slider_label"] = False





    def TT1_Confirm_Btn(self):     # when TT1 Confirm button is clicked
        global confirmed_status   # in order to change global values
        global warn_values
        global vendor_count
        self.TT1_warn.setVisible(False)   # set warn and error Labels invisible
        self.TT1_err.setVisible(False)
        warn_values['TT1_input'] = False   # currently no warning
        textTT1 = self.TT1_input.text()   # get text from input field
        if re.fullmatch(r"\d+", textTT1):  # check if input a positive number only
            if (int(textTT1) > 27):  # if higher than 90 --> unsual
                self.TT1_warn.setVisible(True)   # unsual value warning label visible
                warn_values['TT1_input'] = True  # warning in warn dictionary (in order to later show at calculate button that there are unsual values

            vendor_count = int(textTT1)    # declare value as vendor_count
            self.apply_stylesheet(self.TT1_input, border_color="lime")  # make border green because successfully confirmed
            confirmed_status['TT1_input'] = True   # confirm in confirm-dictionary
            self.TT1_err.setVisible(False)   # set error label invisible
        else:
            self.TT1_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT1_input, border_color="red")    # border red
            self.TT1_err.setVisible(True) # error label visible
        self.TT2_input.setFocus()   # focus on next input field, for faster user experience
        
    def TT2_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global product_count
        self.TT2_warn.setVisible(False)
        self.TT2_err.setVisible(False)
        warn_values['TT2_input'] = False
        textTT2 = self.TT2_input.text()
        if re.fullmatch(r"[1-9]\d*", textTT2):  # check if input a positive number only, not 0
            if (int(textTT2) > 54):  # if higher than 54 --> unsual
                self.TT2_warn.setVisible(True)
                warn_values['TT2_input'] = True

            product_count = int(textTT2)
            self.apply_stylesheet(self.TT2_input, border_color="lime")
            confirmed_status['TT2_input'] = True
            self.TT2_err.setVisible(False)
        else:
            self.TT2_input.clear()  # if input not a positive number or 0
            self.apply_stylesheet(self.TT2_input, border_color="red")
            self.TT2_err.setVisible(True)
        self.TT3_input.setFocus()
        
    def TT3_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global is_chain
        self.TT3_warn.setVisible(False)
        self.TT3_err.setVisible(False)
        warn_values['TT3_input'] = False
        textTT3 = self.TT3_input.text()
        if re.fullmatch(r"\d+", textTT3):  # check if input a positive number only
            if (int(textTT3) > 34):  # if higher than 34 --> unsual
                self.TT3_warn.setVisible(True)
                warn_values['TT3_input'] = True

            is_chain = int(textTT3)
            self.apply_stylesheet(self.TT3_input, border_color="lime")
            confirmed_status['TT3_input'] = True
            self.TT3_err.setVisible(False)
        else:
            self.TT3_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT3_input, border_color="red")
            self.TT3_err.setVisible(True)
        self.TT9_input.setFocus()
    
    def TT4_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global first_order
        self.TT4_warn.setVisible(False)
        self.TT4_err.setVisible(False)
        warn_values['TT4_input'] = False
        textTT4 = self.TT4_input.text()
        if re.fullmatch(r"\d+", textTT4):  # check if input a positive number only
            if (int(textTT4) > 90):  # if higher than 90 --> unsual
                self.TT4_warn.setVisible(True)
                warn_values['TT4_input'] = True

            first_order = int(textTT4)
            self.apply_stylesheet(self.TT4_input, border_color="lime")
            confirmed_status['TT4_input'] = True
            self.TT4_err.setVisible(False)
        else:
            self.TT4_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT4_input, border_color="red")
            self.TT4_err.setVisible(True)
        self.TT5_input.setFocus()
    
    def TT5_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global last_order
        self.TT5_warn.setVisible(False)
        self.TT5_err.setVisible(False)
        warn_values['TT5_input'] = False
        textTT5 = self.TT5_input.text()
        if re.fullmatch(r"\d+", textTT5):  # check if input a positive number only
            if (int(textTT5) > 90):  # if higher than 90 --> unsual
                self.TT5_warn.setVisible(True)
                warn_values['TT5_input'] = True

            last_order = int(textTT5)
            self.apply_stylesheet(self.TT5_input, border_color="lime")
            confirmed_status['TT5_input'] = True
            self.TT5_err.setVisible(False)
        else:
            self.TT5_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT5_input, border_color="red")
            self.TT5_err.setVisible(True)
        self.TT6_input_slider.setFocus()

    def TT6_Confirm_Btn(self):
        global confirmed_status
        global total_expenses
        self.TT6_warn.setVisible(False)
        textTT6 = self.TT6_input_slider.value()   # get text from current slider value

        if (textTT6 > 328):    # if is higher than 328 --> unsual value
            self.TT6_warn.setVisible(True)
            warn_values['TT6_slider'] = True
        total_expenses = textTT6   # save in total_expenses variable
        confirmed_status['TT6_slider_label'] = True
        self.TT6_slider_label.setStyleSheet("""
                    QLabel {
                        font-size: 16px;
                        color: black;
                        background-color: white;
                        border: 2px solid lime;
                        border-radius: 5px;
                    }
                """)
        self.TT7_input_slider.setFocus()
    
    def TT7_Confirm_Btn(self):
        global confirmed_status
        global culinary_variety
        textTT7 = self.TT7_input_slider.value()
        textTT7 = textTT7 / 100   # divide slider value by 100 to get percent, because slider does not work with values between 0 and 1

        culinary_variety = textTT7
        confirmed_status['TT7_slider_label'] = True
        self.TT7_slider_label.setStyleSheet("""
                    QLabel {
                        font-size: 16px;
                        color: black;
                        background-color: white;
                        border: 2px solid lime;
                        border-radius: 5px;
                    }
                """)
        self.calc_cluster_button.setFocus()


    def TT8_Confirm_Btn(self):
        global confirmed_status
        global customer_age_group
        if (self.TT8_combo.currentText() == "---Choose---"):  # if ---Choose--- was chosen --> error label for choosing one
            self.TT8_err.setVisible(True)
            return
        elif (self.TT8_combo.currentText() == "15-17"):  # if this option was chosen --> set final value as 1
            textTT8 = 1
        elif (self.TT8_combo.currentText() == "18-22"):
            textTT8 = 2
        elif (self.TT8_combo.currentText() == "23-28"):
            textTT8 = 3
        elif (self.TT8_combo.currentText() == "29-35"):
            textTT8 = 4
        elif (self.TT8_combo.currentText() == "36-49"):
            textTT8 = 5
        elif (self.TT8_combo.currentText() == "50+"):
            textTT8 = 6

        customer_age_group = textTT8  # save in customer_age_group variable
        confirmed_status['TT8_combo'] = True  #confirm status True
        self.TT8_combo.setStyleSheet("""
                                        QComboBox {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid lime;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        QComboBox:focus {
                                            border: 2px solid blue;
                                        }
                                        QComboBox:editable:on {
                                            border: 2px solid blue;
                                        }
                                        QComboBox:!editable:on {
                                            border: 2px solid blue;
                                        }
                                        QComboBox QAbstractItemView {
                                            background-color: white;
                                            color: black;
                                            border: 2px solid white;
                                            selection-background-color: lightblue;
                                            selection-color: black;
                                        }
                                                    """) # make border green
        self.TT4_input.setFocus()
    
    
    def TT9_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global OrdersPlaced
        self.TT9_warn.setVisible(False)
        self.TT9_err.setVisible(False)
        warn_values['TT9_input'] = False
        textTT9 = self.TT9_input.text()
        if re.fullmatch(r"[1-9]\d*", textTT9):  # check if input a positive number only and not 0
            if (int(textTT9) > 94):  # if higher than 94 --> unsual
                self.TT9_warn.setVisible(True)
                warn_values['TT9_input'] = True

            OrdersPlaced = int(textTT9)
            self.apply_stylesheet(self.TT9_input, border_color="lime")
            confirmed_status['TT9_input'] = True
            self.TT9_err.setVisible(False)
        else:
            self.TT9_input.clear()  # if input not a positive number or 0
            self.apply_stylesheet(self.TT9_input, border_color="red")
            self.TT9_err.setVisible(True)
        self.TT8_combo.setFocus()


    def animate_loading(self):   # for the animation of loading label after clicking on 3d plot
        global is_growing  # a value that changes every 0.5 seconds
        if is_growing:   # if currently is_growing == True
            self.load_label_3d.setStyleSheet("border: 10px solid black;"   #make border and font bigger
                                             "font-size: 45px;")
            is_growing = False   # set is_growing false
        else:   # if is_growing false
            self.load_label_3d.setStyleSheet("border: 5px solid black;"  # make border and font smaller
                                             "font-size: 30px;")
            is_growing = True   # set is_growing true

    def calc_cluster(self):  # method for when the add button to cluster is pressed
        self.UnsualWarn.setVisible(False)  # first set all warn/Error/Pred Label invisible
        self.pred_label.setVisible(False)
        self.FirstLast_err.setVisible(False)
        if all(confirmed_status.values()): # if all fields are confirmed
            if any(warn_values.values()):  # if there is any warning
                self.UnsualWarn.setVisible(True)  # set warnlabel visible
            self.NotConfirmed_err.setVisible(False)  # not confirmed error label invisible(because all values confirmed)

            if first_order > last_order:   # if first_order > last_order --> logic error
                self.FirstLast_err.setVisible(True)   # set error label visible
                self.apply_stylesheet(self.TT4_input, border_color="red")  # borders of first order and last order red
                self.apply_stylesheet(self.TT5_input, border_color="red")
                return  # skip rest of the method of button

            lifetime_days = last_order - first_order    # calculate these values needed for clustering, based on the input values
            avg_per_product = total_expenses / product_count
            avg_per_order = total_expenses / OrdersPlaced
            avg_order_size = product_count / OrdersPlaced
            chain_preference = is_chain / OrdersPlaced
            loyalty_to_venders = vendor_count / OrdersPlaced

            finalList = [vendor_count, product_count, is_chain, first_order, last_order, lifetime_days, total_expenses,
                         avg_per_product, avg_per_order, avg_order_size, culinary_variety, chain_preference, loyalty_to_venders,
                         customer_age_group]   # final list with all values for clustering (metric features)
            print("final_List:", finalList)
            dfDataAll = pd.DataFrame([finalList], columns=metric_features)   # make a dataframe with these values and metric_features as columns
            print("Head:", dfDataAll.head(1))

            dfDataAll = np.log1p(dfDataAll)   # use log function on the values
            print("headAfterLog:", dfDataAll.head(1))

            #scaling:
            dfDataAll[metric_features_scale] = scaler1.transform(dfDataAll[metric_features_scale]) # use scaler 1 for values who get scaled seperately

            group1_values = dfDataAll[group1].values.flatten().reshape(-1, 1)   # reshape values of Scale group 1
            transformed_values = scaler2.transform(group1_values)  # use sacler2 for group 1
            dfDataAll[group1] = transformed_values.reshape(-1, len(group1))  # reshape back

            group2_values = dfDataAll[group2].values.flatten().reshape(-1, 1)   # same for group 2
            transformed_values2 = scaler3.transform(group2_values)
            dfDataAll[group2] = transformed_values2.reshape(-1, len(group2))
            print("headAfterScaling:", dfDataAll.head(1))


            # by calculating ward distance to cluster centroids the new point gets added to one of the clusters
            new_point = dfDataAll.to_numpy()
            ward_distances = []

            if new_point.ndim > 1:  # if dimension too big
                new_point = new_point.squeeze()

            for cluster in np.unique(labels):  # for each cluster
                cluster_points = df[metric_features][labels == cluster]
                n_C = len(cluster_points)

                cluster_center = np.mean(cluster_points, axis=0) # calculate cluster center for each cluster (centroid)

                distance = (n_C / (n_C + 1)) * (euclidean(new_point, cluster_center) ** 2)  # calculate distance between new point and centroid
                ward_distances.append(distance) # add this distance to ward_distances list

            assigned_cluster = np.argmin(ward_distances)    # get the one with lowest distance as assigned cluster
            if(assigned_cluster == 0):  # if it is cluster 0, set this in text of final pred label, and description of cluster
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"do not order a lot, but if then big orders")
            if (assigned_cluster == 1):
                    self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                            f"best spending customers")
            if (assigned_cluster == 2):
                    self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                            f"lowest spending customers")
            if (assigned_cluster == 3):
                    self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                            f"frequently spending regular customers")
            if (assigned_cluster == 4):
                    self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                            f"lower spending regular customers")
            if (assigned_cluster == 5):
                    self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                            f"regular customers who avoid chains")
            self.pred_label.adjustSize()
            self.pred_label.setVisible(True)  # set the final result pred label visible

        else:  # if not all Fields confirmedd
            self.NotConfirmed_err.setVisible(True)  # make error label for not Confirmed visible
            self.pred_label.setVisible(False)  # final pred label invisible
            for key, value in confirmed_status.items():  # make the borders of all not confirmed fields darkblue
                if not value:
                    attribute = getattr(self, key, None)
                    if attribute:
                        self.apply_stylesheet(attribute, border_color="darkblue;", border_px="3px",
                                              border_status="solid")
            return

    def plot_2_1(self):   # method to plot plot_2_1 (from page 2)
        global lastGraph2
        lastGraph2 = 1 # set the last called graph of page 2 as 1

        combo_value = int(self.P2_combo.currentText())   # get the value of combobox (for the cluster that is used)
        fig, ax = plt.subplots(figsize=(8, 8))
        selected_cluster = combo_value
        cluster_data = df_concat[df_concat['labels'] == selected_cluster]

        sns.boxplot(data=cluster_data[metric_features], orient="h", ax=ax)  # create boxplot
        ax.set_title(f"Feature Distribution for Cluster {selected_cluster}")
        ax.set_xlabel("Feature Values")
        fig.tight_layout()

        canvas = FigureCanvas(fig)  # via canvas, show this plot in the label of page 2 to show grapgs
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_2_2(self):   # method to plot plot_2_2 (from page 2)
        global lastGraph2
        lastGraph2 = 2

        combo_value = int(self.P2_combo.currentText())
        fig, ax = plt.subplots(figsize=(8, 8))
        selected_cluster = combo_value
        cluster_mean = df_concat[df_concat['labels'] == selected_cluster][metric_features].mean()

        sns.heatmap(cluster_mean.to_frame(), annot=True, cmap="coolwarm", cbar=False, ax=ax, fmt=".2f")
        ax.set_title(f"Feature Mean Values for Cluster {selected_cluster}")
        ax.set_xlabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_2_3(self):   # method to plot plot_2_3 (from page 2)
        global lastGraph2
        lastGraph2 = 3

        combo_value = int(self.P2_combo.currentText())
        fig, ax = plt.subplots(figsize=(8, 8))
        selected_cluster = combo_value
        cluster_data = df_concat[df_concat['labels'] == selected_cluster][metric_features]

        std_dev = cluster_data.std()
        std_dev.plot(kind='barh', ax=ax, color='darkblue', alpha=0.8)
        ax.set_title(f"Cluster Cohesion (Std Dev) for Cluster {selected_cluster}")
        ax.set_xlabel("Standard Deviation")
        ax.set_ylabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_3_1(self):  # method to plot plot_3_1 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        lastGraphCheck3 = 1   # last called graph of page 3 to 1 (necessary for calling last called plot after checkbox change)
        lastGraph3 = 0   # set last called graph of page 3 to 0 (only set to 1 for plot 3_3 (who needs value from combobox))

        checked_list = []  # to count who many and which checkboxes are checked
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")   # reset border of graphButton 3_2

        if self.C0_checkbox.isChecked(): # count how many and which checkboxes are checked
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if (checked_count == 0):  # if 0 choosen
            self.P3_err.setText("Choose at least one cluster")  # error label visible
            self.apply_stylesheet(self.showGraph3_1, border_color="red")  # border of this button red
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey") # border of other buttons reset
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})    # if > 1, call the plot

            cluster_ids = checked_list
            cluster_means = df_concat.groupby('labels').mean().loc[cluster_ids]

            categories = metric_features
            N = len(categories)

            values = cluster_means.values.T
            values = np.concatenate((values, values[:1, :]), axis=0)

            angles = [n / float(N) * 2 * pi for n in range(N)]
            angles += angles[:1]

            angles = angles[::-1]

            plt.xticks(angles[:-1], categories)

            for idx, cluster_id in enumerate(cluster_ids):
                ax.plot(angles, values[:, idx], label=f'Cluster {cluster_id}')
                ax.fill(angles, values[:, idx], alpha=0.25)

            ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
            ax.set_title("Radar Chart: Comparing Clusters")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()

    def plot_3_2(self):      # method to plot plot_3_2 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        lastGraphCheck3 = 2
        lastGraph3 = 0 # only 1 when graph 3_3 (because needs combobox value)

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if(checked_count != 2):
            self.P3_err.setText("Choose exactly two clusters")
            self.apply_stylesheet(self.showGraph3_2, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            fig, ax = plt.subplots(figsize=(8, 8))
            cluster_ids = checked_list
            cluster_means = df_concat.groupby('labels').mean()

            diff = cluster_means.loc[cluster_ids[1]] - cluster_means.loc[cluster_ids[0]]
            diff.plot(kind='bar', ax=ax, color='darkblue', alpha=0.7)

            ax.axhline(0, color='black', linewidth=1, linestyle='--')
            ax.set_title(f"Feature Differences Between Clusters {cluster_ids[0]} and {cluster_ids[1]}")
            ax.set_ylabel("Difference in Feature Means")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()

    def plot_3_3(self):  # method to plot plot_3_3 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        lastGraphCheck3 = 3
        lastGraph3 = 1  # set as 1 because this graph needs value from combobox

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if (checked_count == 0):
            self.P3_err.setText("Choose at least one cluster")
            self.apply_stylesheet(self.showGraph3_3, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            current_index = self.P3_combo.currentIndex()    # get value from combobox
            fig, ax = plt.subplots(figsize=(8, 8))
            selected_feature = metric_features[current_index]
            cluster_ids = checked_list

            for cluster_id in cluster_ids:
                sns.kdeplot(
                    df_concat[df_concat['labels'] == cluster_id][selected_feature],
                    ax=ax, label=f"Cluster {cluster_id}", fill=True, alpha=0.5
                )

            ax.set_title(f"Distribution Overlap for Feature: {selected_feature}")
            ax.set_xlabel(selected_feature)
            ax.legend()
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()

    def plot_3_4(self):  # method to plot plot_3_4 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        lastGraphCheck3 = 4
        lastGraph3 = 0   # only 1 for 3_3 because needs combobox value

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if(checked_count == 0):
            self.P3_err.setText("Choose at least one cluster")
            self.P3_err.setVisible(True)
            self.apply_stylesheet(self.showGraph3_4, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
        else:
            fig, ax = plt.subplots(figsize=(8, 8))
            cluster_ids = checked_list
            cluster_centers = df_concat.groupby('labels')[metric_features].mean().loc[cluster_ids]

            distances = cdist(
                df_concat[df_concat['labels'].isin(cluster_ids)][metric_features],
                cluster_centers, metric='euclidean'
            )

            distances_df = pd.DataFrame(distances, columns=[f"Cluster {id}" for id in cluster_ids])
            sns.boxplot(data=distances_df, ax=ax)

            ax.set_title("Distance Distribution Between Points and Cluster Centers")
            ax.set_ylabel("Euclidean Distance")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()


    def plot_4_1(self):   # method to plot plot_4_1 (from page 4)
        global lastGraph4
        lastGraph4 = 0  # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)  # loading label (for 3dplot) invisible
        self.graph_label4.setVisible(True)   # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()   # hide label from 3d plot

        fig, ax = plt.subplots(figsize=(8, 8))
        hc_profile = df_concat.groupby('labels').mean().T
        sns.heatmap(hc_profile,center=0, annot=True, cmap="BrBG", fmt=".2f",ax=ax)
        ax.set_xlabel("Cluster Labels")
        ax.set_title("Cluster Profiling:\nHierarchical Clustering with 6 Clusters")
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def plot_4_2(self):  # method to plot plot_4_2 (from page 4)
        global lastGraph4
        lastGraph4 = 0    # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)   # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(8,8))
        sns.countplot(data=df_concat, x='labels')
        ax.set_title("Cluster Sizes")
        ax.set_xlabel("Cluster Labels")
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_3(self):   # method to plot plot_4_3 (from page 4)
        global lastGraph4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)   # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()    # 3d plot label invisible
        pca = PCA(n_components=2)
        reduced_data = pca.fit_transform(df[metric_features])
        cluster_labels = labels
        plt.figure(figsize=(8, 6))
        fig, ax = plt.subplots(figsize=(8, 8))
        for cluster in range(clusters.n_clusters):
            cluster_points = reduced_data[cluster_labels == cluster]
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster}')
        ax.set_xlabel('PCA Component 1')
        ax.set_ylabel('PCA Component 2')
        ax.set_title('Cluster Visualization with PCA')
        ax.legend()
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_4(self):  # method to plot plot_4_4 (from page 4)
        global lastGraph4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True) # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        importances = []

        X = df[metric_features]
        y = labels
        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X, y)

        importances = pd.Series(clf.feature_importances_, index=metric_features).sort_values(ascending=False)

        fig, ax = plt.subplots(figsize=(8, 8))
        importances.plot(kind='bar', ax=ax, color='darkblue', alpha=0.7)
        ax.set_title("Feature Importance Across Clusters")
        ax.set_ylabel("Importance")
        ax.set_xlabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_5(self):    # method to plot plot_4_5 (from page 4)
        global lastGraph4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide() # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

        cluster_ids = cluster_ids_set
        cluster_means = df_concat.groupby('labels').mean().loc[cluster_ids]

        categories = metric_features
        N = len(categories)

        values = cluster_means.values.T
        values = np.concatenate((values, values[:1, :]), axis=0)

        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]

        angles = angles[::-1]

        plt.xticks(angles[:-1], categories)

        for idx, cluster_id in enumerate(cluster_ids):
            ax.plot(angles, values[:, idx], label=f'Cluster {cluster_id}')
            ax.fill(angles, values[:, idx], alpha=0.25)

        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        ax.set_title("Radar Chart: Comparing Clusters")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def plot_4_6(self):      # method to plot plot_4_6 (from page 4)
        global lastGraph4
        lastGraph4 = 1     # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        current_index = self.P4_combo.currentIndex()
        fig, ax = plt.subplots(figsize=(8, 8))
        selected_feature = metric_features[current_index]
        cluster_ids = cluster_ids_set

        for cluster_id in cluster_ids:
            sns.kdeplot(
                df_concat[df_concat['labels'] == cluster_id][selected_feature],
                ax=ax, label=f"Cluster {cluster_id}", fill=True, alpha=0.5
            )

        ax.set_title(f"Distribution Overlap for Feature: {selected_feature}")
        ax.set_xlabel(selected_feature)
        ax.legend()
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def plot_4_7(self):      # method to plot plot_4_7 (from page 4)
        global lastGraph4
        lastGraph4 = 0       # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)   # load label invisible
        self.graph_label4.setVisible(True)  # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(8, 8))
        cluster_ids = cluster_ids_set
        cluster_centers = df_concat.groupby('labels')[metric_features].mean().loc[cluster_ids]

        distances = cdist(
            df_concat[df_concat['labels'].isin(cluster_ids)][metric_features],
            cluster_centers, metric='euclidean'
        )

        distances_df = pd.DataFrame(distances, columns=[f"Cluster {id}" for id in cluster_ids])

        sns.boxplot(data=distances_df, ax=ax)

        ax.set_title("Distance Distribution Between Points and Cluster Centers")
        ax.set_ylabel("Euclidean Distance")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def on_load_finished(self):
        self.load_label_3d.setVisible(False)

    def plot_4_8(self):         # method to plot plot_4_8 (from page 4), 3d Plot
        global lastGraph4
        lastGraph4 = 0       # only 1 for 4_6, because needs value from combobox

        self.load_label_3d.setVisible(False)   # load label invisible (to reset it)
        self.load_label_3d.setVisible(True) # load label visible (until the plot is fully loaded)

        pca = PCA(n_components=3)   # for plot
        principal_components = pca.fit_transform(df[metric_features])
        pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
        pca_df['cluster'] = labels
        fig = px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', title="PCA 3D Plot (PC1, PC2, PC3)",color='cluster',
                            labels={'PC1': 'PC1', 'PC2': 'PC2', 'PC3': 'PC3'})
        html_content = fig.to_html(full_html=False)   # create plot via html (in order to interact with 3d plot)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmp_file:   # save plot as temporary file
            tmp_file.write(html_content.encode('utf-8'))
            temp_html_path = tmp_file.name

        self.web_view.setGeometry(self.graph_label4.geometry())   # create web view label in place of graph_label4
        self.web_view.setFixedSize(800, 800)
        self.web_view.loadFinished.connect(self.on_load_finished)
        self.web_view.load(QUrl.fromLocalFile(temp_html_path))
        self.web_view.show()
        self.graph_label4.setVisible(False)   # make graph label 4 invisible


    def setupUi(self, MainWindow):    # to create buttons, labels etc
        MainWindow.setObjectName("MainWindow")  # create main window
        MainWindow.resize(1920, 1080)
        MainWindow.setStyleSheet("background-color: white;")

        self.stacked_widget = QtWidgets.QStackedWidget(MainWindow)
        MainWindow.setCentralWidget(self.stacked_widget)

        self.page1 = QtWidgets.QWidget()   # create pages (part of stacked_widget)
        self.setupPage1(self.page1)

        self.page2 = QtWidgets.QWidget()
        self.setupPage2(self.page2)

        self.page3 = QtWidgets.QWidget()
        self.setupPage3(self.page3)

        self.page4 = QtWidgets.QWidget()
        self.setupPage4(self.page4)

        self.page5 = QtWidgets.QWidget()
        self.setupPage5(self.page5)

        self.stacked_widget.addWidget(self.page1)   # add pages to stacked_widget
        self.stacked_widget.addWidget(self.page2)
        self.stacked_widget.addWidget(self.page3)
        self.stacked_widget.addWidget(self.page4)
        self.stacked_widget.addWidget(self.page5)

        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        #---------------------

    def setupPage1(self, page):   # method for page 1
        page.setObjectName("Page1")
        page.setStyleSheet("background-color: lightblue;")

        # Label page 1
        self.label1 = QtWidgets.QLabel(page)
        self.label1.setText("Main Page")
        self.label1.setGeometry(QtCore.QRect(820, 40, 200, 80))
        self.label1.setAlignment(QtCore.Qt.AlignCenter)
        self.label1.setStyleSheet("""
                    QLabel {
                         background-color: lightgray;
                         color: black;
                         border: 2px solid white;
                         font-size: 24px;
                         padding: 5px;
                         border-radius: 10px;
                     }
        """)

        # Button to page 2
        self.to_page2_button = QtWidgets.QPushButton(page)
        self.to_page2_button.setText("Insights in one cluster")
        self.to_page2_button.setGeometry(QtCore.QRect(550, 250, 300, 150))
        self.to_page2_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 24px;
                                            padding: 25px;
                                            
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 24px;
                                            padding: 25px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page2_button.clicked.connect(self.go_to_page2)

        # Button to page 3
        self.to_page3_button = QtWidgets.QPushButton(page)
        self.to_page3_button.setText("Compare clusters")
        self.to_page3_button.setGeometry(QtCore.QRect(1000, 250, 300, 150))
        self.to_page3_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 24px;
                                            padding: 25px;
                                            
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 24px;
                                            padding: 25px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page3_button.clicked.connect(self.go_to_page3)

        # Button to page 4
        self.to_page4_button = QtWidgets.QPushButton(page)
        self.to_page4_button.setText("Insights into all\n clusters")
        self.to_page4_button.setGeometry(QtCore.QRect(550, 500, 300, 150))
        self.to_page4_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 24px;
                                            padding: 25px;
                                            
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 24px;
                                            padding: 25px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page4_button.clicked.connect(self.go_to_page4)

        # Button to page 5
        self.to_page5_button = QtWidgets.QPushButton(page)
        self.to_page5_button.setText("Connect new entry\nto cluster")
        self.to_page5_button.setGeometry(QtCore.QRect(1000, 500, 300, 150))
        self.to_page5_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 24px;
                                            padding: 25px;
                                            
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 24px;
                                            padding: 25px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page5_button.clicked.connect(self.go_to_page5)




    def go_to_page1(self):
        self.stacked_widget.setCurrentWidget(self.page1)

    def setupPage2(self, page):
        page.setObjectName("Page2")
        page.setStyleSheet("background-color: lightblue;")

        # Label Info Cluster page 2
        self.labelInfoCluster2 = QtWidgets.QLabel(page)
        self.labelInfoCluster2.setText(
            "Cluster 0: dont order a lot, but big orders\nCluster 1: best spending customers\nCluster 2: lowest spending"
            " customers\nCluster 3: frequently spending regular customers\nCluster 4: lower spending regular customers\n"
            "Cluster 5: regular customers who avoid chains")
        self.labelInfoCluster2.setGeometry(QtCore.QRect(50, 650, 500, 130))
        self.labelInfoCluster2.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 24px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.labelInfoCluster2.adjustSize()

        # Label for page 2
        self.label2 = QtWidgets.QLabel(page)
        self.label2.setText("Insights in one cluster")
        self.label2.setGeometry(QtCore.QRect(450, 50, 300, 30))
        self.label2.setStyleSheet("""
                            QLabel {
                                 background-color: lightgray;
                                 color: black;
                                 border: 2px solid white;
                                 font-size: 28px;
                                 padding: 5px;
                                 border-radius: 10px;
                             }
                """)
        self.label2.adjustSize()

        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setGeometry(QtCore.QRect(50, 150, 200, 50))
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # Label for combo
        self.combo_label2 = QtWidgets.QLabel(page)
        self.combo_label2.setGeometry(750, 245, 100, 100)
        self.combo_label2.setText("Choose Cluster")
        self.combo_label2.setStyleSheet("border: 0px solid black;"
                                        "font-size: 16px;")
        self.combo_label2.adjustSize()

        # ----- 2 Combo Box -----
        self.P2_combo = QComboBox(page)
        self.P2_combo.addItem("0")
        self.P2_combo.addItem("1")
        self.P2_combo.addItem("2")
        self.P2_combo.addItem("3")
        self.P2_combo.addItem("4")
        self.P2_combo.setGeometry(750, 270, 130, 30)
        self.P2_combo.setObjectName("2_combo")
        self.P2_combo.setStyleSheet("""
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                                """)

        self.P2_combo.view().setMinimumWidth(390)
        self.P2_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P2_combo"))

        # Button Show Graph 2_1
        self.showGraph2_1 = QtWidgets.QPushButton(page)
        self.showGraph2_1.setText("Cluster boxplot")
        self.showGraph2_1.setGeometry(QtCore.QRect(750, 350, 200, 50))
        self.showGraph2_1.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph2_1.clicked.connect(self.plot_2_1)

        # Button Show Graph 2_2
        self.showGraph2_2 = QtWidgets.QPushButton(page)
        self.showGraph2_2.setText("Cluster heatmap")
        self.showGraph2_2.setGeometry(QtCore.QRect(750, 450, 200, 50))
        self.showGraph2_2.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph2_2.clicked.connect(self.plot_2_2)

        # Button Show Graph 2_3
        self.showGraph2_3 = QtWidgets.QPushButton(page)
        self.showGraph2_3.setText("Cluster Coherence")
        self.showGraph2_3.setGeometry(QtCore.QRect(750, 550, 200, 50))
        self.showGraph2_3.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph2_3.clicked.connect(self.plot_2_3)

        # Label for graph 2
        self.graph_label2 = QtWidgets.QLabel(page)
        self.graph_label2.setGeometry(1000, 100, 800, 800)
        self.graph_label2.setStyleSheet("border: 1px solid black;")

    def go_to_page2(self):
        self.stacked_widget.setCurrentWidget(self.page2)
    
    def setupPage3(self, page):
        page.setObjectName("Page3")
        page.setStyleSheet("background-color: lightblue;")

        self.labelInfoCluster3 = QtWidgets.QLabel(page)
        self.labelInfoCluster3.setText(
            "Cluster 0: dont order a lot, but big orders\nCluster 1: best spending customers\nCluster 2: lowest spending"
            " customers\nCluster 3: frequently spending regular customers\nCluster 4: lower spending regular customers\n"
            "Cluster 5: regular customers who avoid chains")
        self.labelInfoCluster3.setGeometry(QtCore.QRect(50, 650, 500, 130))
        self.labelInfoCluster3.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 24px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.labelInfoCluster3.adjustSize()

        self.label3 = QtWidgets.QLabel(page)
        self.label3.setText("Compare clusters")
        self.label3.setGeometry(QtCore.QRect(450, 50, 300, 30))
        self.label3.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 28px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.label3.adjustSize()


        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setGeometry(QtCore.QRect(50, 150, 200, 50))
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # ---- The Page3 Error Label
        self.P3_err = QtWidgets.QLabel(page)
        self.P3_err.setText("----------------------------------")
        self.P3_err.setGeometry(QtCore.QRect(590, 200, 230, 30))
        self.P3_err.setVisible(False)
        self.P3_err.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid red;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                        """)
        self.P3_err.adjustSize()



        # ----- 3 Combo Box -----
        self.P3_combo = QComboBox(page)
        self.P3_combo.addItems(metric_features)
        self.P3_combo.setGeometry(450, 510, 180, 40)
        self.P3_combo.setObjectName("3_combo")
        self.P3_combo.setStyleSheet("""
                                    background-color: white;
                                    color: black;
                                    border: 2px solid white;
                                    font-size: 16px;
                                    padding: 5px;
                                    border-radius: 10px;
                                    """)

        self.P3_combo.view().setMinimumWidth(200)
        self.P3_combo.view().setMinimumHeight(280)
        self.P3_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P3_combo"))

        # ----- C0 Checkbox -----
        self.C0_checkbox = QtWidgets.QCheckBox(page)
        self.C0_checkbox.setText("C0")
        self.C0_checkbox.setGeometry(QtCore.QRect(400, 250, 74, 50))
        self.C0_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C0_checkbox.setStyleSheet("""
                            QCheckBox {
                                font-size: 16px;
                                color: black;
                                padding: 0px;
                                border-radius: 10px;
                            }
                            QCheckBox::indicator {
                                width: 30px;
                                height: 30px;
                            }
                        """)

        # ----- C1 Checkbox -----
        self.C1_checkbox = QtWidgets.QCheckBox(page)
        self.C1_checkbox.setText("C1")
        self.C1_checkbox.setGeometry(QtCore.QRect(500, 250, 74, 50))
        self.C1_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C1_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        font-size: 16px;
                                        color: black;
                                        padding: 0px;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)

        # ----- C2 Checkbox -----
        self.C2_checkbox = QtWidgets.QCheckBox(page)
        self.C2_checkbox.setText("C2")
        self.C2_checkbox.setGeometry(QtCore.QRect(600, 250, 74, 50))
        self.C2_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C2_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        font-size: 16px;
                                        color: black;
                                        padding: 0px;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)

        # ----- C3 Checkbox -----
        self.C3_checkbox = QtWidgets.QCheckBox(page)
        self.C3_checkbox.setText("C3")
        self.C3_checkbox.setGeometry(QtCore.QRect(700, 250, 74, 50))
        self.C3_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C3_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        font-size: 16px;
                                        color: black;
                                        padding: 0px;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)

        # ----- C4 Checkbox -----
        self.C4_checkbox = QtWidgets.QCheckBox(page)
        self.C4_checkbox.setText("C4")
        self.C4_checkbox.setGeometry(QtCore.QRect(800, 250, 74, 50))
        self.C4_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C4_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        font-size: 16px;
                                        color: black;
                                        padding: 0px;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)

        # ----- C5 Checkbox -----
        self.C5_checkbox = QtWidgets.QCheckBox(page)
        self.C5_checkbox.setText("C5")
        self.C5_checkbox.setGeometry(QtCore.QRect(900, 250, 74, 50))
        self.C5_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C5_checkbox.setStyleSheet("""
                                        QCheckBox {
                                            font-size: 16px;
                                            color: black;
                                            padding: 0px;
                                            border-radius: 10px;
                                        }
                                        QCheckBox::indicator {
                                            width: 30px;
                                            height: 30px;
                                        }
                                    """)

        # Button Show Graph 3_1
        self.showGraph3_1 = QtWidgets.QPushButton(page)
        self.showGraph3_1.setText("Radar Chart")
        self.showGraph3_1.setObjectName("button_3_1")
        self.showGraph3_1.setGeometry(QtCore.QRect(450, 350, 200, 50))
        self.showGraph3_1.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph3_1.clicked.connect(self.plot_3_1)

        # Button Show Graph 3_2
        self.showGraph3_2 = QtWidgets.QPushButton(page)
        self.showGraph3_2.setText("Feature Difference Bar chart")
        self.showGraph3_2.setObjectName("button_3_2")
        self.showGraph3_2.setGeometry(QtCore.QRect(750, 350, 200, 50))
        self.showGraph3_2.setStyleSheet("""
                                        QPushButton {
                                            font-size: 14px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 14px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph3_2.clicked.connect(self.plot_3_2)

        # Button Show Graph 3_3
        self.showGraph3_3 = QtWidgets.QPushButton(page)
        self.showGraph3_3.setText("Distribution Overlap Plot")
        self.showGraph3_3.setObjectName("button_3_3")
        self.showGraph3_3.setGeometry(QtCore.QRect(450, 450, 200, 50))
        self.showGraph3_3.setStyleSheet("""
                                        QPushButton {
                                            font-size: 16px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 16px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph3_3.clicked.connect(self.plot_3_3)

        # Button Show Graph 3_4
        self.showGraph3_4 = QtWidgets.QPushButton(page)
        self.showGraph3_4.setText("Distance Plot")
        self.showGraph3_4.setObjectName("button_3_4")
        self.showGraph3_4.setGeometry(QtCore.QRect(750, 450, 200, 50))
        self.showGraph3_4.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph3_4.clicked.connect(self.plot_3_4)

        # Label for graph 3
        self.graph_label3 = QtWidgets.QLabel(page)
        self.graph_label3.setGeometry(1000, 100, 800, 800)
        self.graph_label3.setStyleSheet("border: 1px solid black;")


    def go_to_page3(self):
        self.stacked_widget.setCurrentWidget(self.page3)

    def setupPage4(self, page):

        page.setObjectName("Page4")
        page.setStyleSheet("background-color: lightblue;")

        self.labelInfoCluster4 = QtWidgets.QLabel(page)
        self.labelInfoCluster4.setText(
            "Cluster 0: dont order a lot, but big orders\nCluster 1: best spending customers\nCluster 2: lowest spending"
            " customers\nCluster 3: frequently spending regular customers\nCluster 4: lower spending regular customers\n"
            "Cluster 5: regular customers who avoid chains")
        self.labelInfoCluster4.setGeometry(QtCore.QRect(50, 650, 500, 130))
        self.labelInfoCluster4.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 24px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.labelInfoCluster4.adjustSize()

        self.label4 = QtWidgets.QLabel(page)
        self.label4.setText("Insights in all clusters")
        self.label4.setGeometry(QtCore.QRect(450, 50, 300, 30))
        self.label4.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 28px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.label4.adjustSize()

        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setGeometry(QtCore.QRect(50, 150, 200, 50))
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # ----- 4 Combo Box -----
        self.P4_combo = QComboBox(page)
        self.P4_combo.addItems(metric_features)
        self.P4_combo.setGeometry(710, 620, 180, 40)
        self.P4_combo.setObjectName("4_combo")
        self.P4_combo.setStyleSheet("""
                                    background-color: white;
                                    color: black;
                                    border: 2px solid white;
                                    font-size: 16px;
                                    padding: 5px;
                                    border-radius: 10px;
                                    """)

        self.P4_combo.view().setMinimumWidth(200)
        self.P4_combo.view().setMinimumHeight(280)
        self.P4_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P4_combo"))

        # Button Show Graph 4_1
        self.showGraph4_1 = QtWidgets.QPushButton(page)
        self.showGraph4_1.setText("Cluster Infos")
        self.showGraph4_1.setGeometry(QtCore.QRect(400, 260, 200, 50))
        self.showGraph4_1.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_1.clicked.connect(self.plot_4_1)

        # Button Show Graph 4_2
        self.showGraph4_2 = QtWidgets.QPushButton(page)
        self.showGraph4_2.setText("Cluster Sizes")
        self.showGraph4_2.setGeometry(QtCore.QRect(400, 360, 200, 50))
        self.showGraph4_2.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_2.clicked.connect(self.plot_4_2)

        # Button Show Graph 4_3
        self.showGraph4_3 = QtWidgets.QPushButton(page)
        self.showGraph4_3.setText("Cluster Overwiew")
        self.showGraph4_3.setGeometry(QtCore.QRect(400, 460, 200, 50))
        self.showGraph4_3.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_3.clicked.connect(self.plot_4_3)


        # Button Show Graph 4_4
        self.showGraph4_4 = QtWidgets.QPushButton(page)
        self.showGraph4_4.setText("Feature Importance p Cluster")
        self.showGraph4_4.setGeometry(QtCore.QRect(700, 460, 200, 50))
        self.showGraph4_4.setStyleSheet("""
                                        QPushButton {
                                            font-size: 14px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 14px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_4.clicked.connect(self.plot_4_4)

        # Button Show Graph 4_5
        self.showGraph4_5 = QtWidgets.QPushButton(page)
        self.showGraph4_5.setText("Radar Chart")
        self.showGraph4_5.setGeometry(QtCore.QRect(700, 260, 200, 50))
        self.showGraph4_5.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_5.clicked.connect(self.plot_4_5)

        # Button Show Graph 4_6
        self.showGraph4_6 = QtWidgets.QPushButton(page)
        self.showGraph4_6.setText("Distribution Overlap Plot")
        self.showGraph4_6.setGeometry(QtCore.QRect(700, 560, 200, 50))
        self.showGraph4_6.setStyleSheet("""
                                        QPushButton {
                                            font-size: 16px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 16px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_6.clicked.connect(self.plot_4_6)

        # Button Show Graph 4_7
        self.showGraph4_7 = QtWidgets.QPushButton(page)
        self.showGraph4_7.setText("Distance Plot")
        self.showGraph4_7.setGeometry(QtCore.QRect(700, 360, 200, 50))
        self.showGraph4_7.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_7.clicked.connect(self.plot_4_7)

        # Button Show Graph 4_8
        self.showGraph4_8 = QtWidgets.QPushButton(page)
        self.showGraph4_8.setText("3D Cluster PCA")
        self.showGraph4_8.setGeometry(QtCore.QRect(400, 560, 200, 50))
        self.showGraph4_8.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.showGraph4_8.clicked.connect(self.plot_4_8)

        #Loading label 3d
        self.load_label_3d = QtWidgets.QLabel(page)
        self.load_label_3d.setGeometry(670, 700, 300, 80)
        self.load_label_3d.setText("... Loading ...")
        self.load_label_3d.setAlignment(Qt.AlignCenter)
        self.load_label_3d.setStyleSheet("border: 5px solid black;"
                                         "font-size: 30px;")
        self.load_label_3d.setVisible(False)

        #Timer for animation
        self.timer3d = QTimer(page)
        self.timer3d.timeout.connect(self.animate_loading)
        self.timer3d.start(500)

        self.web_view = QWebEngineView(page)

        # Label for graph 4
        self.graph_label4 = QtWidgets.QLabel(page)
        self.graph_label4.setGeometry(1000, 100, 800, 800)
        self.graph_label4.setStyleSheet("border: 1px solid black;")



    def go_to_page4(self):
        self.stacked_widget.setCurrentWidget(self.page4)
    
    def setupPage5(self, page):
        page.setObjectName("Page5")
        page.setStyleSheet("background-color: lightblue;")

        self.label5 = QtWidgets.QLabel(page)
        self.label5.setText("Connect new entry to cluster")
        self.label5.setGeometry(QtCore.QRect(450, 50, 300, 30))
        self.label5.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         font-size: 28px;
                                         padding: 5px;
                                         border-radius: 10px;
                                     }
                        """)
        self.label5.adjustSize()

        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setGeometry(QtCore.QRect(50, 50, 200, 50))
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # Calculate Button
        self.calc_cluster_button = QtWidgets.QPushButton(page)
        self.calc_cluster_button.setText("Add to cluster")
        self.calc_cluster_button.setGeometry(QtCore.QRect(950, 700, 200, 50))
        self.calc_cluster_button.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.calc_cluster_button.clicked.connect(self.calc_cluster)

        # Label for explanation
        self.labelExp = QtWidgets.QLabel(page)
        self.labelExp.setText("by calculating ward distance\nto cluster centroids")
        self.labelExp.setGeometry(QtCore.QRect(950, 760, 300, 30))
        self.labelExp.setStyleSheet("font-size: 18px; color: black;")
        self.labelExp.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.labelExp.adjustSize()




        # ----- TT1 Label -----
        self.TT1_label = QtWidgets.QLabel(page)
        self.TT1_label.setText("Vendor Count:")
        self.TT1_label.setGeometry(QtCore.QRect(50, 150, 130, 30))
        self.TT1_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                """)
        self.TT1_label.adjustSize()

        # ----- TT1 Input field -----
        self.TT1_input = QtWidgets.QLineEdit(page)
        self.TT1_input.setGeometry(QtCore.QRect(50, 200, 140, 30))
        self.TT1_input.setObjectName("TT1_input")
        self.TT1_input.setPlaceholderText("0-27, mean:2.9")
        self.TT1_input.returnPressed.connect(lambda: self.TT1_Confirm_Btn())
        self.TT1_input.textChanged.connect(lambda: self.on_text_change_input("TT1_label", "TT1_input"))
        self.TT1_input.setStyleSheet("""
                                    QLineEdit {
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                    QLineEdit:focus {
                                        border: 2px solid blue;
                                    }
                                """)

        # ----- The TT1 Confirm Button -----
        self.TT1_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT1_Confirm_Btn())
        self.TT1_Confirm.setText("Confirm")
        self.TT1_Confirm.setGeometry(QtCore.QRect(50, 250, 130, 30))
        self.TT1_Confirm.setObjectName("btn_TT1_Confirm")
        self.TT1_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)

        # ---- The TT1 Error Label
        self.TT1_err = QtWidgets.QLabel(page)
        self.TT1_err.setText("Only positive numbers")
        self.TT1_err.setGeometry(QtCore.QRect(50, 300, 230, 30))
        self.TT1_err.setVisible(False)
        self.TT1_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                                """)
        self.TT1_err.adjustSize()

        # ---- The TT1 Warn Label
        self.TT1_warn = QtWidgets.QLabel(page)
        self.TT1_warn.setText("Unsual Values, might\nlead to different results")
        self.TT1_warn.setGeometry(QtCore.QRect(50, 300, 180, 40))
        self.TT1_warn.setVisible(False)
        self.TT1_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT1_warn.adjustSize()





        # ----- TT2 Label -----
        self.TT2_label = QtWidgets.QLabel(page)
        self.TT2_label.setText("Product Count:")
        self.TT2_label.setGeometry(QtCore.QRect(350, 150, 130, 30))
        self.TT2_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT2_label.adjustSize()

        # ----- TT2 Input field -----
        self.TT2_input = QtWidgets.QLineEdit(page)
        self.TT2_input.setGeometry(QtCore.QRect(350, 200, 130, 30))
        self.TT2_input.textChanged.connect(lambda: self.on_text_change_input("TT2_label", "TT2_input"))
        self.TT2_input.returnPressed.connect(lambda: self.TT2_Confirm_Btn())
        self.TT2_input.setObjectName("TT2_input")
        self.TT2_input.setPlaceholderText("1-54, mean:5")
        self.TT2_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)

        # ----- The TT2 Confirm Button -----
        self.TT2_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT2_Confirm_Btn())
        self.TT2_Confirm.setText("Confirm")
        self.TT2_Confirm.setGeometry(QtCore.QRect(350, 250, 130, 30))
        self.TT2_Confirm.setObjectName("btn_TT2_Confirm")
        self.TT2_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)



        # ---- The TT2 Error Label
        self.TT2_err = QtWidgets.QLabel(page)
        self.TT2_err.setText("Only positive numbers > 0")
        self.TT2_err.setGeometry(QtCore.QRect(350, 300, 230, 30))
        self.TT2_err.setVisible(False)
        self.TT2_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT2_err.adjustSize()

        # ---- The TT2 Warn Label
        self.TT2_warn = QtWidgets.QLabel(page)
        self.TT2_warn.setText("Unsual Values, might\nlead to different results")
        self.TT2_warn.setGeometry(QtCore.QRect(350, 300, 180, 40))
        self.TT2_warn.setVisible(False)
        self.TT2_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT2_warn.adjustSize()


        # ----- TT3 Label -----
        self.TT3_label = QtWidgets.QLabel(page)
        self.TT3_label.setText("is Chain:")
        self.TT3_label.setGeometry(QtCore.QRect(650, 150, 130, 30))
        self.TT3_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT3_label.adjustSize()

        # ----- TT3 Input field -----
        self.TT3_input = QtWidgets.QLineEdit(page)
        self.TT3_input.setGeometry(QtCore.QRect(650, 200, 140, 30))
        self.TT3_input.textChanged.connect(lambda: self.on_text_change_input("TT3_label", "TT3_input"))
        self.TT3_input.returnPressed.connect(lambda: self.TT3_Confirm_Btn())
        self.TT3_input.setObjectName("TT3_input")
        self.TT3_input.setPlaceholderText("0-34, mean:2.5")
        self.TT3_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        # ----- The TT3 Confirm Button -----
        self.TT3_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT3_Confirm_Btn())
        self.TT3_Confirm.setText("Confirm")
        self.TT3_Confirm.setGeometry(QtCore.QRect(650, 250, 130, 30))
        self.TT3_Confirm.setObjectName("btn_TT3_Confirm")
        self.TT3_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)


        # ---- The TT3 Error Label
        self.TT3_err = QtWidgets.QLabel(page)
        self.TT3_err.setText("Only positive numbers")
        self.TT3_err.setGeometry(QtCore.QRect(650, 300, 230, 30))
        self.TT3_err.setVisible(False)
        self.TT3_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)

        # ---- The TT3 Warn Label
        self.TT3_warn = QtWidgets.QLabel(page)
        self.TT3_warn.setText("Unsual Values, might\nlead to different results")
        self.TT3_warn.setGeometry(QtCore.QRect(650, 300, 180, 40))
        self.TT3_warn.setVisible(False)
        self.TT3_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT3_warn.adjustSize()


        # ----- TT4 Label -----
        self.TT4_label = QtWidgets.QLabel(page)
        self.TT4_label.setText("first Order:")
        self.TT4_label.setGeometry(QtCore.QRect(350, 400, 130, 30))
        self.TT4_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT4_label.adjustSize()

        # ----- TT4 Input field -----
        self.TT4_input = QtWidgets.QLineEdit(page)
        self.TT4_input.setGeometry(QtCore.QRect(350, 450, 130, 30))
        self.TT4_input.textChanged.connect(lambda: self.on_text_change_input("TT4_label", "TT4_input"))
        self.TT4_input.returnPressed.connect(lambda: self.TT4_Confirm_Btn())
        self.TT4_input.setObjectName("TT4_input")
        self.TT4_input.setPlaceholderText("0-90, mean:28")
        self.TT4_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)

        # ----- The TT4 Confirm Button -----
        self.TT4_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT4_Confirm_Btn())
        self.TT4_Confirm.setText("Confirm")
        self.TT4_Confirm.setGeometry(QtCore.QRect(350, 500, 130, 30))
        self.TT4_Confirm.setObjectName("btn_TT4_Confirm")
        self.TT4_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)


        # ---- The TT4 Error Label
        self.TT4_err = QtWidgets.QLabel(page)
        self.TT4_err.setText("Only positive numbers")
        self.TT4_err.setGeometry(QtCore.QRect(350, 550, 230, 30))
        self.TT4_err.setVisible(False)
        self.TT4_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT4_err.adjustSize()

        # ---- The TT4 Warn Label
        self.TT4_warn = QtWidgets.QLabel(page)
        self.TT4_warn.setText("Unsual Values, might\nlead to different results")
        self.TT4_warn.setGeometry(QtCore.QRect(350, 550, 180, 40))
        self.TT4_warn.setVisible(False)
        self.TT4_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT4_warn.adjustSize()


        # ----- TT5 Label -----
        self.TT5_label = QtWidgets.QLabel(page)
        self.TT5_label.setText("last Order:")
        self.TT5_label.setGeometry(QtCore.QRect(650, 400, 130, 30))
        self.TT5_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT5_label.adjustSize()

        # ----- TT5 Input field -----
        self.TT5_input = QtWidgets.QLineEdit(page)
        self.TT5_input.setGeometry(QtCore.QRect(650, 450, 130, 30))
        self.TT5_input.textChanged.connect(lambda: self.on_text_change_input("TT5_label", "TT5_input"))
        self.TT5_input.returnPressed.connect(lambda: self.TT5_Confirm_Btn())
        self.TT5_input.setObjectName("TT5_input")
        self.TT5_input.setPlaceholderText("0-90, mean:63")
        self.TT5_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)

        # ----- The TT5 Confirm Button -----
        self.TT5_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT5_Confirm_Btn())
        self.TT5_Confirm.setText("Confirm")
        self.TT5_Confirm.setGeometry(QtCore.QRect(650, 500, 130, 30))
        self.TT5_Confirm.setObjectName("btn_TT5_Confirm")
        self.TT5_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)



        # ---- The TT5 Error Label
        self.TT5_err = QtWidgets.QLabel(page)
        self.TT5_err.setText("Only positive numbers")
        self.TT5_err.setGeometry(QtCore.QRect(650, 550, 230, 30))
        self.TT5_err.setVisible(False)
        self.TT5_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT5_err.adjustSize()

        # ---- The TT5 Warn Label
        self.TT5_warn = QtWidgets.QLabel(page)
        self.TT5_warn.setText("Unsual Values, might\nlead to different results")
        self.TT5_warn.setGeometry(QtCore.QRect(650, 550, 180, 40))
        self.TT5_warn.setVisible(False)
        self.TT5_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT5_warn.adjustSize()



        # ----- TT6 Label -----
        self.TT6_label = QtWidgets.QLabel(page)
        self.TT6_label.setText("Total expenses:")
        self.TT6_label.setGeometry(QtCore.QRect(230, 650, 130, 30))
        self.TT6_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                """)
        self.TT6_label.adjustSize()

        # ----- TT6 Input Slider -----
        self.TT6_input_slider = QtWidgets.QSlider(page)
        self.TT6_input_slider.setGeometry(QtCore.QRect(180, 700, 230, 30))
        self.TT6_input_slider.setOrientation(QtCore.Qt.Horizontal)
        self.TT6_input_slider.setMinimum(0)
        self.TT6_input_slider.setMaximum(500)
        self.TT6_input_slider.setValue(0)
        self.TT6_input_slider.setObjectName("TT6_input_slider")
        self.TT6_input_slider.valueChanged.connect(self.update_label_slide_TT6)
        self.TT6_input_slider.setStyleSheet("""
            QSlider::groove:horizontal {
                border: 2px solid #bbb;
                background: white;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::sub-page:horizontal {
                background: blue;
                border: 2px solid #777;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::add-page:horizontal {
                background: #ddd;
                border: 2px solid #777;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::handle:horizontal {
                background: #333;
                border: 2px solid #555;
                width: 18px;
                height: 18px;
                margin: -5px 0; /* handle is placed outside groove */
                border-radius: 9px;
            }
            QSlider::handle:horizontal:hover {
                background: #444;
            }
            QSlider::handle:focus {
                background-color: lightgrey;
            }
            QSlider::sub-page:horizontal:focus {
                background: blue;
                border: 2px solid lightgrey;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::add-page:horizontal:focus {
                background: #ddd;
                border: 2px solid lightgrey;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::sub-page:horizontal:disabled {
                background: #bbb;
                border-color: #999;
            }
            QSlider::add-page:horizontal:disabled {
                background: #eee;
                border-color: #999;
            }
            QSlider::handle:horizontal:disabled {
                background: #eee;
                border: 2px solid #aaa;
            }
        """)

        self.TT6_slider_label = QtWidgets.QLabel(page)
        self.TT6_slider_label.setGeometry(QtCore.QRect(230, 735, 130, 30))
        self.TT6_slider_label.setAlignment(QtCore.Qt.AlignCenter)
        self.TT6_slider_label.setText("0  -  mean:35")
        self.TT6_slider_label.setObjectName("TT6_slider_label")
        self.TT6_slider_label.setStyleSheet("""
            QLabel {
                font-size: 16px;
                color: black;
                background-color: white;
                border: 2px solid gray;
                border-radius: 5px;
            }
        """)

        # ----- The TT6 Confirm Button -----
        self.TT6_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT6_Confirm_Btn())
        self.TT6_Confirm.setText("Confirm")
        self.TT6_Confirm.setGeometry(QtCore.QRect(230, 785, 130, 30))
        self.TT6_Confirm.setObjectName("btn_TT6_Confirm")
        self.TT6_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)

        # ---- The TT6 Warn Label
        self.TT6_warn = QtWidgets.QLabel(page)
        self.TT6_warn.setText("Unsual Values, might\nlead to different results")
        self.TT6_warn.setGeometry(QtCore.QRect(180, 850, 180, 40))
        self.TT6_warn.setVisible(False)
        self.TT6_warn.setStyleSheet("""
                                        QLabel {
                                            background-color: lightgray;
                                            color: black;
                                            border: 2px solid orange;
                                            font-size: 16px;
                                            padding: 5px;
                                            border-radius: 10px;
                                        }
                                    """)
        self.TT6_warn.adjustSize()

        # ----- TT7 Label -----
        self.TT7_label = QtWidgets.QLabel(page)
        self.TT7_label.setText("Culinary Variety:")
        self.TT7_label.setGeometry(QtCore.QRect(580, 650, 130, 30))
        self.TT7_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT7_label.adjustSize()


        # ----- TT7 Input Slider -----
        self.TT7_input_slider = QtWidgets.QSlider(page)
        self.TT7_input_slider.setGeometry(QtCore.QRect(530, 700, 230, 30))
        self.TT7_input_slider.setOrientation(QtCore.Qt.Horizontal)
        self.TT7_input_slider.setMinimum(0)
        self.TT7_input_slider.setMaximum(100)
        self.TT7_input_slider.setValue(0)
        self.TT7_input_slider.setObjectName("TT7_input_slider")
        self.TT7_input_slider.valueChanged.connect(self.update_label_slide_TT7)
        self.TT7_input_slider.setStyleSheet("""
                    QSlider::groove:horizontal {
                        border: 2px solid #bbb;
                        background: white;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::sub-page:horizontal {
                        background: blue;
                        border: 2px solid #777;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::add-page:horizontal {
                        background: #ddd;
                        border: 2px solid #777;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::handle:horizontal {
                        background: #333;
                        border: 2px solid #555;
                        width: 18px;
                        height: 18px;
                        margin: -5px 0; /* handle is placed outside groove */
                        border-radius: 9px;
                    }
                    QSlider::handle:focus {
                        background-color: lightgrey;
                    }
                    QSlider::sub-page:horizontal:focus {
                        background: blue;
                        border: 2px solid lightgrey;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::add-page:horizontal:focus {
                        background: #ddd;
                        border: 2px solid lightgrey;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::handle:horizontal:hover {
                        background: #444;
                    }
                    QSlider::sub-page:horizontal:disabled {
                        background: #bbb;
                        border-color: #999;
                    }
                    QSlider::add-page:horizontal:disabled {
                        background: #eee;
                        border-color: #999;
                    }
                    QSlider::handle:horizontal:disabled {
                        background: #eee;
                        border: 2px solid #aaa;
                    }
                """)

        self.TT7_slider_label = QtWidgets.QLabel(page)
        self.TT7_slider_label.setGeometry(QtCore.QRect(580, 735, 130, 30))
        self.TT7_slider_label.setAlignment(QtCore.Qt.AlignCenter)
        self.TT7_slider_label.setText("0  -  mean:0.14")
        self.TT7_slider_label.setObjectName("TT7_slider_label")
        self.TT7_slider_label.setStyleSheet("""
                    QLabel {
                        font-size: 16px;
                        color: black;
                        background-color: white;
                        border: 2px solid gray;
                        border-radius: 5px;
                    }
                """)

        # ----- The TT7 Confirm Button -----
        self.TT7_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT7_Confirm_Btn())
        self.TT7_Confirm.setText("Confirm")
        self.TT7_Confirm.setGeometry(QtCore.QRect(580, 785, 130, 30))
        self.TT7_Confirm.setObjectName("btn_TT7_Confirm")
        self.TT7_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)

        # ----- TT8 Label -----
        self.TT8_label = QtWidgets.QLabel(page)
        self.TT8_label.setText("customer Age:")
        self.TT8_label.setGeometry(QtCore.QRect(50, 400, 130, 30))
        self.TT8_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: 16px;
                                        padding: 5px;
                                        border-radius: 10px;
                                    }
                                """)
        self.TT8_label.adjustSize()

        # ----- TT8 Combo Box -----
        self.TT8_combo = QComboBox(page)
        self.TT8_combo.addItem("---Choose---")
        self.TT8_combo.addItem("15-17")
        self.TT8_combo.addItem("18-22")
        self.TT8_combo.addItem("23-28")
        self.TT8_combo.addItem("29-35")
        self.TT8_combo.addItem("36-49")
        self.TT8_combo.addItem("50+")
        self.TT8_combo.setGeometry(50, 450, 130, 30)
        self.TT8_combo.setObjectName("TT8_combo")
        self.TT8_combo.setStyleSheet("""
                        QComboBox {
                                background-color: white;
                                color: black;
                                border: 2px solid white;
                                font-size: 16px;
                                padding: 5px;
                                border-radius: 10px;
                            }
                        QComboBox:focus {
                            border: 2px solid blue;
                        }
                        QComboBox:editable:on {
                            border: 2px solid blue;
                        }
                        QComboBox:!editable:on {
                            border: 2px solid blue;
                        }
                        QComboBox QAbstractItemView {
                                    background-color: white;
                                    color: black;
                                    border: 2px solid white;
                                    selection-background-color: lightblue;
                                    selection-color: black;
                        }
                                    """)

        self.TT8_combo.view().setMinimumWidth(120)
        self.TT8_combo.currentIndexChanged.connect(lambda: self.on_combobox_change("TT8_combo", "TT8_err"))

        # ----- The TT8 Confirm Button -----
        self.TT8_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT8_Confirm_Btn())
        self.TT8_Confirm.setText("Confirm")
        self.TT8_Confirm.setGeometry(QtCore.QRect(50, 500, 130, 30))
        self.TT8_Confirm.setObjectName("btn_TT8_Confirm")
        self.TT8_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)

        # ---- The TT8 Error Label
        self.TT8_err = QtWidgets.QLabel(page)
        self.TT8_err.setText("Please Choose one")
        self.TT8_err.setGeometry(QtCore.QRect(50, 550, 180, 40))
        self.TT8_err.setVisible(False)
        self.TT8_err.setStyleSheet("""
                                                    QLabel {
                                                        background-color: lightgray;
                                                        color: black;
                                                        border: 2px solid red;
                                                        font-size: 16px;
                                                        padding: 5px;
                                                        border-radius: 10px;
                                                    }
                                                """)
        self.TT8_err.adjustSize()

        # ----- TT9 Label -----
        self.TT9_label = QtWidgets.QLabel(page)
        self.TT9_label.setText("Total Number of Orders placed:")
        self.TT9_label.setGeometry(QtCore.QRect(950, 150, 130, 30))
        self.TT9_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT9_label.adjustSize()

        # ----- TT9 Input field -----
        self.TT9_input = QtWidgets.QLineEdit(page)
        self.TT9_input.setGeometry(QtCore.QRect(950, 200, 140, 30))
        self.TT9_input.textChanged.connect(lambda: self.on_text_change_input("TT9_label", "TT9_input"))
        self.TT9_input.returnPressed.connect(lambda: self.TT9_Confirm_Btn())
        self.TT9_input.setObjectName("TT9_input")
        self.TT9_input.setPlaceholderText("1-94, mean:4.3")
        self.TT9_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)

        # ----- The TT9 Confirm Button -----
        self.TT9_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT9_Confirm_Btn())
        self.TT9_Confirm.setText("Confirm")
        self.TT9_Confirm.setGeometry(QtCore.QRect(950, 250, 130, 30))
        self.TT9_Confirm.setObjectName("btn_TT9_Confirm")
        self.TT9_Confirm.setStyleSheet("""
                                        QPushButton {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            font-size: 22px;
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)

        # ---- The TT9 Error Label
        self.TT9_err = QtWidgets.QLabel(page)
        self.TT9_err.setText("Only positive numbers > 0")
        self.TT9_err.setGeometry(QtCore.QRect(950, 300, 230, 30))
        self.TT9_err.setVisible(False)
        self.TT9_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                font-size: 16px;
                                                padding: 5px;
                                                border-radius: 10px;
                                            }
                                        """)
        self.TT9_err.adjustSize()

        # ---- The TT9 Warn Label
        self.TT9_warn = QtWidgets.QLabel(page)
        self.TT9_warn.setText("Unsual Values, might\nlead to different results")
        self.TT9_warn.setGeometry(QtCore.QRect(950, 300, 180, 40))
        self.TT9_warn.setVisible(False)
        self.TT9_warn.setStyleSheet("""
                                                QLabel {
                                                    background-color: lightgray;
                                                    color: black;
                                                    border: 2px solid orange;
                                                    font-size: 16px;
                                                    padding: 5px;
                                                    border-radius: 10px;
                                                }
                                            """)
        self.TT9_warn.adjustSize()

        

        # ----- Prediction Label -----
        self.pred_label = QtWidgets.QLabel(page)
        self.pred_label.setText("No Prediction made yet")
        self.pred_label.setGeometry(QtCore.QRect(1100, 450, 400, 380))
        self.pred_label.setVisible(False)
        self.pred_label.setStyleSheet("""
                                                QLabel {
                                                    background-color: lightgray;
                                                    color: black;
                                                    border: 2px solid white;
                                                    font-size: 22px;
                                                    padding: 5px;
                                                    border-radius: 10px;
                                                }

                                            """)
        self.pred_label.adjustSize()


        # ---- The NotConfirmed Error Label
        self.NotConfirmed_err = QtWidgets.QLabel(page)
        self.NotConfirmed_err.setText("Some entries are not confirmed")
        self.NotConfirmed_err.setGeometry(QtCore.QRect(1100, 450, 250, 130))
        self.NotConfirmed_err.setVisible(False)
        self.NotConfirmed_err.setStyleSheet("""
                                                   QLabel {
                                                       background-color: darkblue;
                                                       color: white;
                                                       border: 6px solid;
                                                       border-color: darkblue;
                                                       font-size: 22px;
                                                       padding: 5px;
                                                       border-radius: 10px;
                                                   }
                                               """)
        self.NotConfirmed_err.adjustSize()

        # ---- The First Order > Last Order Error Label
        self.FirstLast_err = QtWidgets.QLabel(page)
        self.FirstLast_err.setText("First Order cant be bigger than last order")
        self.FirstLast_err.setGeometry(QtCore.QRect(1100, 450, 250, 130))
        self.FirstLast_err.setVisible(False)
        self.FirstLast_err.setStyleSheet("""
                                           QLabel {
                                               background-color: darkblue;
                                               color: white;
                                               border: 6px solid;
                                               border-color: red;
                                               font-size: 22px;
                                               padding: 5px;
                                               border-radius: 10px;
                                           }
                                       """)
        self.FirstLast_err.adjustSize()

        # ---- The Unsual Warn Label
        self.UnsualWarn = QtWidgets.QLabel(page)
        self.UnsualWarn.setText("Some entries have unsual values,\nresults may be unexcpected")
        self.UnsualWarn.setGeometry(QtCore.QRect(1100, 525, 250, 130))
        self.UnsualWarn.setVisible(False)
        self.UnsualWarn.setStyleSheet("""
                                               QLabel {
                                                   background-color: lightgray;
                                                   color: black;
                                                   border: 6px solid orange;
                                                   font-size: 22px;
                                                   padding: 5px;
                                                   border-radius: 10px;
                                               }
                                           """)
        self.UnsualWarn.adjustSize()


        # ---- Info Calculate Label
        self.calc_info_label = QtWidgets.QLabel(page)
        self.calc_info_label.setText(
            "INFO: Other values will be calculated based on these:\n\nlifetime_days = last_order - first_order\n"
            "avg_per_product = total_expenses / product_count\navg_per_order = total_expenses / OrdersPlaced\n"
            "avg_order_size = product_count / OrdersPlaced\nchain_preference = is_chain / OrdersPlaced\n"
            "loyalty_to_venders = vendor_count / OrdersPlaced")
        self.calc_info_label.setGeometry(QtCore.QRect(1350, 150, 160, 50))
        self.calc_info_label.setStyleSheet("""
                                               QLabel {
                                                   background-color: lightgray;
                                                   color: black;
                                                   border: 2px solid black;
                                                   font-size: 18px;
                                                   text-align: center;
                                                   padding: 5px;
                                                   border-radius: 10px;
                                               }
                                           """)
        self.calc_info_label.adjustSize()


    def go_to_page5(self):
        self.stacked_widget.setCurrentWidget(self.page5)

    # -------------------------------------------------------------------------------------------------------------------

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("Interface", "Interface"))


In [4]:
def runMain():  # method to run the interface
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()

    sys.exit(app.exec_())

In [ ]:
runMain()  # run the interface